In [ ]:
import pandas as pd
df = pd.read_csv( './drive/MyDrive/Data/Task_2.tsv', sep = '\t', names=['paragraph_id','article_id','paragraph', 'keyword','country_code','span_start','span_end','span_text','category_label',  'number_of_annotators_agreeing_on_that_label'] ).iloc[3:,:] 
df.head()

In [ ]:
!pip install sentencepiece transformers


In [ ]:
df_final = pd.DataFrame(columns=['paragraph'])
df_final.paragraph=df.paragraph.unique()
df_final

In [ ]:
y = pd.read_csv('./drive/MyDrive/Data/task_2_labels.txt', sep = ' ', names=[ 'id','l1','l2','l3','l4','l5','l6','l7']).iloc[0:,1:].to_numpy()
y

In [6]:
import numpy as np
np.sum(y, axis = 0)
wt_array = np.sum(y, axis = 0)
wt_array

array([716, 196, 224, 230, 197, 469,  40])

In [ ]:
import torch

from transformers import RobertaTokenizer, RobertaModel
import pandas as pd
df = pd.read_csv('./drive/MyDrive/Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )

df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

# Importing the libraries needed
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel

                 

df_final.shape
#df_final.columns
nclasses = 7


MAX_LEN= 512
TRAIN_BS = 8
VALID_BS = 8
EPOCHS = 1
LR =  1e-05
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

class PclData(Dataset):
    def __init__(self, dataframe, targets, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targets = targets
        print(self.len); print(self.data)
        
    def __getitem__(self, index):# function compulsory
        sent = self.data.paragraph[index]
       # sent = " ".join(sent.split())
        inputs = self.tokenizer.encode_plus(
        sent, 
        None,
        add_special_tokens = True,
        max_length=self.max_len,
        padding = 'max_length',
        return_token_type_ids = True,
        truncation=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return { 'ids': torch.tensor(ids, dtype=torch.long),
               'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
                
               }
    
    def __len__(self):
        return self.len
        

In [ ]:
df_final.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset, y_train, y_val = train_test_split(df_final,  y, test_size = 0.20, random_state= 0)
train_dataset= train_dataset.reset_index(drop=True)
val_dataset= val_dataset.reset_index(drop=True)
#test_dataset = df_test

print("FULL Dataset: {}".format(df_final.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))

training_set = PclData(train_dataset,y_train, tokenizer, MAX_LEN)
val_set = PclData(val_dataset,y_val, tokenizer, MAX_LEN)
#test_set= PclData(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BS,
               'shuffle':True,
               'num_workers': 2}
val_params = {'batch_size': VALID_BS,
              'shuffle': False,
              'num_workers': 2}

trainloader = DataLoader(training_set, **train_params)
valloader = DataLoader(val_set, **val_params)
#testloader = DataLoader(test_set, **val_params)

In [ ]:
363 in train_dataset.index

False

In [68]:
for i, e in enumerate (trainloader):
  if i==99:
    print(e)


{'ids': tensor([[    0,   108,   318,  ...,     1,     1,     1],
        [    0, 27298,     8,  ...,     1,     1,     1]]), 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'targets': tensor([[0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 1, 0]])}


In [9]:
class RobertaModelClass (torch.nn.Module):
    def __init__(self, nclasses):
        super(RobertaModelClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained('roberta-base')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, nclasses)

    def forward(self, input_ids, attention_mask):
        with torch.autograd.no_grad():
            output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaModelClass(nclasses)
model.to(device)

In [17]:
class_weights=torch.FloatTensor(wt_array)

In [18]:
loss_function = torch.nn.BCEWithLogitsLoss(weight=class_weights)
optimizer = torch.optim.Adam(params =model.parameters(), lr=LR)
def calculate_acc(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct


In [19]:
#loss_function(torch.tensor([[0,1,0,0,1,0,1],[0,0,1,0,0,0,1]]).float(),torch.tensor([[0,1,0,0,1,0,1],[0,0,1,0,0,0,1]]).float())

tensor(186.2587)

In [11]:
 '''loss_function = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(params =model.parameters(), lr=LR)
def calculate_acc(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct'''


In [20]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/textclassify_experiment_1')


In [11]:
from sklearn.metrics import accuracy_score
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.train()
    for _, data in tqdm(enumerate(trainloader, 0)):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)
        
        outputs = model(ids, mask)
        loss = loss_function(outputs.float(), targets.float())
        tr_loss += loss.item()
        big_val= (torch.sigmoid(outputs.data)>.5).float()
        #print(big_val, targets)
        n_correct += accuracy_score(big_val, targets)
        
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if _ % 50 == 0:
            
            loss_step = tr_loss/nb_tr_steps
            acc_step = (n_correct*100)/nb_tr_examples
            print(f'Training Loss per 50 steps: {loss_step}, Training Accuracy: {acc_step}')
            writer.add_scalar('training_loss', loss_step, epoch*len(trainloader) +_)
            
            
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    print(f'Total Accuracy Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Training Loss Epoch: {epoch_loss}, Training Accuracy Epoch: {epoch_acc}')
    valid(model, valloader)
    return

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# Validation

def valid(model, valloader):
    model.eval()
    n_correct =0; n_wrong =0; total=0; tr_loss =0; nb_tr_steps =0; nb_tr_examples=0
    y_pred, y_true = [],[]
    with torch.no_grad():
        for _, data in enumerate(valloader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            
            outputs = model(ids, mask)
            loss = loss_function(outputs.float(), targets.float())
            tr_loss += loss.item()
            big_val= (torch.sigmoid(outputs.data)>.5).float()
            n_correct += accuracy_score(big_val, targets)

            y_true.extend(targets.cpu().detach().numpy())
      
            y_pred.extend(torch.argmax(outputs, dim=1).cpu().detach().numpy())
            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)
            
            if _ % 100 == 0:
                loss_step = tr_loss/nb_tr_steps
                acc_step = (n_correct*100)/nb_tr_examples
                print(f'Validation Loss per 100 steps: {loss_step}, Validation Accuracy per: {acc_step}')   
        
    epoch_loss = tr_loss/nb_tr_steps
    epoch_acc = (n_correct*100)/nb_tr_examples
    print(f'Validation Loss Epoch: {epoch_loss}, Validation Accuracy Epoch: {epoch_acc}')
    
    return epoch_acc, y_true, y_pred

In [21]:
for epoch in range(EPOCHS):
    train(epoch)

1it [00:16, 16.48s/it]

Training Loss per 50 steps: 210.382080078125, Training Accuracy: 0.0


51it [13:35, 15.92s/it]

Training Loss per 50 steps: 202.57570663152956, Training Accuracy: 0.24509803921568626


100it [26:23, 15.84s/it]

Total Accuracy Epoch 0: 0.551007556675063
Training Loss Epoch: 200.71433685302733, Training Accuracy Epoch: 0.551007556675063


Validation Loss per 100 steps: 199.36941528320312, Validation Accuracy per: 0.0
Validation Loss Epoch: 197.5749938964844, Validation Accuracy Epoch: 2.207465900933238


In [44]:
O = torch.tensor([[-0.0296,  0.0850, -0.0057, -0.1630,  0.0371,  0.0408, -0.2133],
        [-0.0577,  0.0481,  0.0702, -0.1217, -0.0902, -0.1280,  0.0807],
        [-0.0510,  0.0819,  0.0864, -0.0274,  0.0847, -0.0118, -0.0684],
        [-0.1678, -0.0129,  0.0421, -0.3505, -0.0362, -0.0037, -0.0363],
        [-0.0596,  0.0678,  0.0669, -0.1484, -0.0905, -0.0592,  0.0393],
        [-0.0811,  0.0376,  0.0361, -0.0984,  0.1248, -0.1142,  0.0044],
        [-0.1048,  0.0958,  0.1091, -0.0912,  0.0316, -0.0225,  0.0171],
        [-0.0591,  0.1196, -0.0192, -0.1452, -0.0287, -0.0437, -0.0278]])
T = torch.tensor([[1., 1., 0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 1., 0.],
        [1., 1., 0., 0., 0., 0., 0.]])

big_val= (torch.sigmoid(O.data)>.5).float()
(big_val>.5).float()

tensor([[0., 1., 0., 0., 1., 1., 0.],
        [0., 1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 0., 1., 0., 1.],
        [0., 1., 1., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0.]])

In [31]:
acc, y_true, y_pred = valid(model, valloader)

Validation Loss per 100 steps: 199.36941528320312, Validation Accuracy per: 0.0
Validation Loss Epoch: 197.5749938964844, Validation Accuracy Epoch: 2.207465900933238


In [41]:
'''from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
accuracy = confusion_matrix(y_true, y_pred)
print(accuracy)'''
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

ValueError: ignored

In [40]:
 (y_pred).value_counts()

AttributeError: ignored

In [ ]:
f =open('bert_roberta_small_test_1.txt', 'w')
for i in y_pred:
  print(i, file = f )
f.close()
